<a href="https://colab.research.google.com/github/xtchen64/virtual-doctor-chatbot/blob/main/notebooks/llm_api_call.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# empty cell